In [44]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [45]:
# Train Phase transformations
train_transforms = transforms.Compose([
                                      #  transforms.Resize((28, 28)),
                                      #  transforms.ColorJitter(brightness=0.10, contrast=0.1, saturation=0.10, hue=0.1),
                                      
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.1307,), (0.3081,)) # The mean and std have to be sequences (e.g., tuples), therefore you should add a comma after the values. 
                                       # Note the difference between (0.1307) and (0.1307,)
                                       ])

# Test Phase transformations
test_transforms = transforms.Compose([
                                      #  transforms.Resize((28, 28)),
                                      #  transforms.ColorJitter(brightness=0.10, contrast=0.1, saturation=0.10, hue=0.1),
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.1307,), (0.3081,))
                                       ])


In [46]:
train = datasets.MNIST('./data', train=True, download=True, transform=train_transforms)
test = datasets.MNIST('./data', train=False, download=True, transform=test_transforms)

In [47]:
SEED = 1

# CUDA?
cuda = torch.cuda.is_available()
print("CUDA Available?", cuda)

# For reproducibility
torch.manual_seed(SEED)

if cuda:
    torch.cuda.manual_seed(SEED)

# dataloader arguments - something you'll fetch these from cmdprmt
dataloader_args = dict(shuffle=True, batch_size=128, num_workers=4, pin_memory=True) if cuda else dict(shuffle=True, batch_size=64)

# train dataloader
train_loader = torch.utils.data.DataLoader(train, **dataloader_args)

# test dataloader
test_loader = torch.utils.data.DataLoader(test, **dataloader_args)

CUDA Available? True


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [55]:
class Net(nn.Module):

  def __init__(self):
    super(Net, self).__init__()
    
    #convolutuonla block 1
    self.convblock1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU()
    )#input_size = 28, #output_size = 26, RF = 3
    self.convblock2 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU()
    ) #input_size = 26, #output_size = 24, RF = 5
    self.convblock3 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU()
    ) #input_size = 24, #output_size = 22, RF = 7
    
    # transition block
    self.maxpool1 = nn.MaxPool2d(2,2) #input_size = 22, #output_size = 11, RF = 8
    self.convblock4 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU()
    )#input_size = 11, #output_size = 9, RF = 12
    self.convblock5 = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU()
    )#input_size = 9, #output_size = 7, RF = 16

    #convolutuonla block 2
    self.convblock6 = nn.Sequential(
            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU()
    )#input_size = 7, #output_size = 5, RF = 20
    self.convblock7 = nn.Sequential(
            nn.Conv2d(in_channels=512, out_channels=1024, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU()
    )#input_size = 5, #output_size = 3, RF = 24

   #output block
    self.convblock8 = nn.Sequential(
            nn.Conv2d(in_channels=1024, out_channels=10, kernel_size=(3,3), padding=0, bias=False),
            
    )#input_size = 3, #output_size = 1, RF = 28

  def forward(self,input):

    x = self.convblock3(self.convblock2(self.convblock1(input)))

    x = self.convblock4(self.maxpool1(x))

    x = self.convblock8(self.convblock7(self.convblock6(self.convblock5(x))))

  

    return F.log_softmax(x)



In [56]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             144
              ReLU-2           [-1, 16, 26, 26]               0
            Conv2d-3           [-1, 32, 24, 24]           4,608
              ReLU-4           [-1, 32, 24, 24]               0
            Conv2d-5           [-1, 64, 22, 22]          18,432
              ReLU-6           [-1, 64, 22, 22]               0
         MaxPool2d-7           [-1, 64, 11, 11]               0
            Conv2d-8            [-1, 128, 9, 9]          73,728
              ReLU-9            [-1, 128, 9, 9]               0
           Conv2d-10            [-1, 256, 7, 7]         294,912
             ReLU-11            [-1, 256, 7, 7]               0
           Conv2d-12            [-1, 512, 5, 5]       1,179,648
             ReLU-13            [-1, 512, 5, 5]               0
           Conv2d-14           [-1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:57: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [59]:
from tqdm import tqdm

train_losses = []
test_losses = []
train_acc = []
test_acc = []

def train(model, device, train_loader, optimizer, epoch):
  model.train()
  #pbar = tqdm(train_loader)
  correct = 0
  processed = 0
  for batch_idx, (data, target) in enumerate(train_loader):
    # get samples
    data, target = data.to(device), target.to(device)
    #print("target size:",target.size())
    #target = target.squeeze(1)
    #print("target size2:",target.size())
    # Init
    optimizer.zero_grad()
    # In PyTorch, we need to set the gradients to zero before starting to do backpropragation because PyTorch accumulates the gradients on subsequent backward passes. 
    # Because of this, when you start your training loop, ideally you should zero out the gradients so that you do the parameter update correctly.

    # Predict
    y_pred = model(data)
    #print("y_pred size:",y_pred.size())
    y_pred = y_pred.squeeze(-1)
    y_pred = y_pred.squeeze(-1)
    #print("y_pred size2:",y_pred.size())
    # Calculate loss
    loss = F.nll_loss(y_pred, target)
    train_losses.append(loss)

    # Backpropagation
    loss.backward()
    optimizer.step()

    # Update pbar-tqdm
    
    pred = y_pred.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
    correct += pred.eq(target.view_as(pred)).sum().item()
    processed += len(data)

    #pbar.set_description(desc= f'Loss={loss.item()} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
    train_acc.append(100*correct/processed)
  return (100*correct/processed)

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            output = output.squeeze(-1)
            output = output.squeeze(-1)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)
    
    """
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    """
    test_acc.append(100. * correct / len(test_loader.dataset))
    return (100. * correct / len(test_loader.dataset))

In [60]:
model =  Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
EPOCHS = 14
for epoch in range(EPOCHS):
    #print("EPOCH:", epoch)
    train_accuracy = train(model, device, train_loader, optimizer, epoch)
    test_accuracy = test(model, device, test_loader)
    print("epoch no: ",epoch+1," train_accuracy: ",train_accuracy," test_accuracy: ",test_accuracy)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:57: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


epoch no:  1  train_accuracy:  67.98333333333333  test_accuracy:  96.47
epoch no:  2  train_accuracy:  97.355  test_accuracy:  97.86
epoch no:  3  train_accuracy:  98.32833333333333  test_accuracy:  98.01
epoch no:  4  train_accuracy:  98.72833333333334  test_accuracy:  98.81
epoch no:  5  train_accuracy:  99.07  test_accuracy:  98.84
epoch no:  6  train_accuracy:  99.25666666666666  test_accuracy:  99.13
epoch no:  7  train_accuracy:  99.415  test_accuracy:  98.97
epoch no:  8  train_accuracy:  99.50333333333333  test_accuracy:  99.05
epoch no:  9  train_accuracy:  99.59166666666667  test_accuracy:  99.1
epoch no:  10  train_accuracy:  99.62  test_accuracy:  99.1
epoch no:  11  train_accuracy:  99.71333333333334  test_accuracy:  99.08
epoch no:  12  train_accuracy:  99.76333333333334  test_accuracy:  99.01
epoch no:  13  train_accuracy:  99.77  test_accuracy:  99.03
epoch no:  14  train_accuracy:  99.75666666666666  test_accuracy:  99.21


Target :


1.  Setting up the colab file
2.   Setting basic working code
3. Setting training and testing loop

Results :


1.  Parameters - 6.38M
2.   Best Trai accuracy - 99.77 (within 14 epochs)
3.  Best Test accuracy - 99.21

Analysis :


1.  Overfitting in model
2. Training accuracy would have been higher if trained for higher epochs
3. Model is heavy for recognising digits. Need to decrease parameters.



